## Understanding MySQL `EXPLAIN`

The MySQL `EXPLAIN` command is a powerful tool that helps us understand how MySQL plans to execute a query. It provides detailed information about what MySQL will do behind the scenes, which is very useful for learning about indexing, query optimization, and database performance. Key columns in the `EXPLAIN` output include:

- **`select_type`**: Indicates the type of `SELECT` being performed. For example, whether it is a **simple** select (just retrieving rows) or a **primary/select with subqueries**. This helps differentiate between straightforward queries and more complex ones.

- **`table`**: The name of the table being accessed by the query.

- **`partitions`**: Shows which partitions of a table (if the table is partitioned) are being scanned. Partitioning allows MySQL to split data across different sections for performance.

- **`type`**: Often the most important column. It tells you the join type MySQL is using and whether it will scan the full table or use an index. Common values include:
  - `ALL` → full table scan (slow)
  - `index` → full index scan
  - `ref` → index lookup by non-unique key
  - `eq_ref` → unique index lookup
  - `const`/`system` → very fast, single-row lookups

- **`possible_keys`**: Shows which indexes MySQL **could potentially** use to execute the query. These are candidate indexes, but MySQL may not actually use them.

- **`key`**: The index MySQL **actually chose** to use for the query.

- **`key_len`**: The length of the key MySQL decided to use. This indicates how much of the index is being leveraged, which can affect performance.

- **`ref`**: Shows which column or constant is being used to match rows against the key. For example, it could be a column in the query or a constant value from a `WHERE` clause.

- **`rows`**: An estimate of the number of rows MySQL expects to examine to execute the query. This is based on table statistics.

- **`filtered`**: A percentage estimate of how many rows will actually match the `WHERE` conditions after filtering. This helps understand MySQL's selectivity predictions.

- **`Extra`**: Additional notes about the query execution. Common examples:
  - `Using where` → MySQL is applying a `WHERE` filter after reading rows
  - `Using index` → MySQL is retrieving all information from the index without accessing the table itself
  - `Using temporary` → MySQL needs a temporary table (e.g., for `GROUP BY` or `ORDER BY`)
  - `Using filesort` → MySQL must sort results outside of the index

In this notebook, we will explore these fields in more depth and observe how they change depending on different queries and indexing strategies. Understanding `EXPLAIN` is key to writing efficient SQL queries and leveraging indexes properly.

A good article is this: https://planetscale.com/blog/how-read-mysql-explains

### Database initialization and table information

In [1]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

init_db(20000)

execute_query("SHOW TABLES", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Products table")
execute_query("DESC products", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Brands table")
execute_query("DESC availability", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Brands table")
execute_query("DESC brands", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Categories table")
execute_query("DESC categories", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Colors table")
execute_query("DESC colors", database=get_database_name(), print_as_df=True, show_metrics=False)



Created empty database
Created empty tables
Starting to insert data

Starting insert into 'products' (20000 rows)...
Finished inserting 20000 rows into 'products' in 0.38s.


,Tables_in_test_database
0,availability
1,brands
2,categories
3,colors
4,products


Products table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,,None,
2,description,text,YES,,None,
3,brand_id,int,YES,MUL,None,
4,category_id,int,YES,MUL,None,
5,price,float,YES,,None,
6,currency,varchar(10),YES,,None,
7,stock,int,YES,,None,
8,ean,varchar(50),YES,,None,
9,color_id,int,YES,MUL,None,


Brands table


...

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,UNI,None,


Categories table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,UNI,None,


Colors table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(100),YES,UNI,None,


# select_type

#### **Case 1**: Simple `SELECT` with a `WHERE` condition

This query is straightforward:

* We select all product names where the price is less than 100.

* Since there are no joins, no subqueries, and no grouping, the query plan is very simple.

* The optimizer just needs to scan the `products` table and filter rows based on the condition `price < 100`.

In the `EXPLAIN` output, you’ll see only one row, because only one query block is involved.
This is what MySQL calls a SIMPLE query.

*side note, this would be the exact same without the where*

In [2]:
query = """
EXPLAIN SELECT name FROM products WHERE price < 100;
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,None,None,None,None,20000,33.33,Using where


### Case 2: Query with a subquery

Here, we add a subquery:

* The inner query `(SELECT AVG(price) FROM products)` calculates the average price.

* The outer query then compares each product’s price to that average.

In the `EXPLAIN` output:

* The first row is labeled `PRIMARY`, meaning it’s the main query block.

* The second row is labeled `SUBQUERY`, representing the inner `SELECT`.

Even though the first row looks similar to the one in Case 1, it’s now called `PRIMARY` because the overall statement contains more than one query block.

👉 Rule of thumb:

* If there’s only one query block, you’ll see `SIMPLE`.

* If there are multiple blocks (because of subqueries, unions, etc.), the outermost one becomes `PRIMARY`.

In [ ]:
query = """
EXPLAIN
SELECT name
FROM products
WHERE price < (SELECT AVG(price) FROM products);
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,PRIMARY,products,None,ALL,None,None,None,None,20000,33.33,Using where
1,2,SUBQUERY,products,None,ALL,None,None,None,None,20000,100.00,None


### Case 3: Derived table

This looks different from Case 2:

* Instead of a subquery in the `WHERE` clause, here we build a derived table (sometimes called a *subselect* in the *`FROM`* clause).

* The inner query groups products by category and calculates the average price per category.

* That result set (the grouped averages) is treated like a temporary table named `sub`.

* The outer query then simply selects from this derived table.

In the `EXPLAIN` output:

* The inner query is labeled DERIVED.

* This means MySQL must materialize the results of the inner query into a temporary result set before the outer query can run.

* The outer query shows up as `PRIMARY`, since it’s the main block.

👉 Difference between subquery and derived table:

* Subquery: Appears in a `WHERE`, `HAVING`, or `SELECT` clause; usually produces a scalar value (like one number) or is used in an `IN` condition.

* Derived table: Appears in the `FROM` clause and produces a table-like result set that the outer query can work with.

In [4]:
query = """
EXPLAIN
SELECT sub.avg_price
FROM (
    SELECT category_id, AVG(price) AS avg_price
    FROM products
    GROUP BY category_id
) AS sub;
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,PRIMARY,<derived2>,None,ALL,None,None,None,None,20000,100.0,None
1,2,DERIVED,products,None,index,category_id,category_id,5,None,20000,100.0,None


### Case 4: Query with `UNION`

Here we combine two separate result sets into one using UNION:

* The first query block selects product names where price < 50.

* The second query block selects product names where price > 500.

* UNION merges them and removes duplicates by default (if you want duplicates allowed, you would use UNION ALL).

In the EXPLAIN output:

* Each SELECT statement gets its own row: one for the first query and one for the second.

* The outermost block is labeled PRIMARY.

* The inner blocks show as UNION queries, since their results are combined into the final output.

👉 Key point:

* UNION forces MySQL to execute each query separately and then merge the results.

* That’s why you see multiple blocks in the execution plan.

In [5]:
query = """
EXPLAIN
SELECT name FROM products WHERE price < 50
UNION
SELECT name FROM products WHERE price > 500;
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,PRIMARY,products,None,ALL,None,None,None,None,20000.0,33.33,Using where
1,2,UNION,products,None,ALL,None,None,None,None,20000.0,33.33,Using where
2,3,UNION RESULT,"<union1,2>",None,ALL,None,None,None,None,NaN,NaN,Using temporary


### Case 5: Correlated subquery

This query introduces a correlated subquery:

* The outer query (p1) selects product names.

* For each product, the inner query (p2) calculates the average price within the same category as that product (p2.category_id = p1.category_id).

* The outer query then checks whether the product’s price is less than that category’s average.

In the EXPLAIN output:

* The outer query appears as PRIMARY.

* The inner query appears as DEPENDENT SUBQUERY (instead of just SUBQUERY).

* This label means the subquery depends on a value from the outer query (p1.category_id).

MySQL must run the subquery once per row of the outer query, which can be much slower than a simple subquery.

👉 Key point:

* Regular subquery: Can often be executed once and reused.

* Correlated subquery: Must be re-evaluated for every row of the outer query, which is less efficient.

In [6]:
query = """
EXPLAIN
SELECT p1.name
FROM products p1
WHERE p1.price < (
    SELECT AVG(p2.price)
    FROM products p2
    WHERE p2.category_id = p1.category_id
);
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,PRIMARY,p1,None,ALL,None,None,None,None,20000,100.0,Using where
1,2,DEPENDENT SUBQUERY,p2,None,ALL,category_id,None,None,None,20000,10.0,Using where


# type

In [7]:
query = """
EXPLAIN SELECT 1+1;
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,None,None,None,None,None,None,None,None,None,No tables used


In [8]:
query = """
EXPLAIN SELECT * FROM products WHERE id = 123;
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,const,PRIMARY,PRIMARY,4,const,1,100.0,None


In [9]:
query = """
EXPLAIN
SELECT p.name, c.name
FROM products p
JOIN categories c ON p.category_id = c.id;
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,p,None,ALL,category_id,None,None,None,20000,100.0,Using where
1,1,SIMPLE,c,None,eq_ref,PRIMARY,PRIMARY,4,test_database.p.category_id,1,100.0,None


In [10]:
query = """
EXPLAIN SELECT * FROM products WHERE category_id = 5;
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,category_id,category_id,5,const,568,100.0,None


In [ ]:
query = "CREATE FUTEXT INDEX idx_name ON products(name);"

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

query = """
EXPLAIN SELECT * FROM products WHERE MATCH(name) AGAINST('Charger');
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=True)

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,fulltext,idx_name,idx_name,0,const,1,100.0,Using where; Ft_hints: sorted


[QUERY METRICS] 1 rows fetched, 0.87 ms


In [12]:
query = """
CREATE INDEX idx_price ON products(price);
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

query = """
EXPLAIN SELECT price FROM products ORDER BY price;
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,index,None,idx_price,5,None,19792,100.0,Using index


In [13]:
query = """
EXPLAIN SELECT * FROM products WHERE name LIKE '%%Charger%%';
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,None,None,None,None,19792,11.11,Using where
